In [1]:
import numpy as np
import pandas as pd

In [2]:
hospital_file = 'YAIB-cohorts/data/mortality24/eicu/above2000.txt'
df = pd.read_csv(hospital_file, header=None)

In [3]:
hospital_list = df.values.flatten()

In [10]:
save_dict = {} 
for h in hospital_list: 
    load_data = np.load(f"yaib_logs/eicu/Mortality24/LogisticRegression/train{h}-test{h}/data.npz", allow_pickle=True)
    all_features = np.concatenate([load_data['train'].item()['features'], 
                                  load_data['val'].item()['features'], 
                                  load_data['test'].item()['features']])
    all_labels = np.concatenate([load_data['train'].item()['labels'], 
                              load_data['val'].item()['labels'], 
                              load_data['test'].item()['labels']])
    save_dict[str(h)] = {}
    save_dict[str(h)]['features'] = all_features
    save_dict[str(h)]['labels']= all_labels

In [11]:
np.savez("processed_data/all_hospitals.npz", **save_dict)

In [15]:
load_data = np.load("processed_data/all_hospitals.npz", allow_pickle=True)
load_data['73']

array({'features': array([[-0.60222611, -0.43733733, -0.24494304, ...,  0.95820366,
        -0.44720557,  3.        ],
       [ 0.        ,  0.        ,  0.        , ..., -0.39895073,
        -0.88033197,  3.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.95820366,
         0.20603423,  3.        ],
       ...,
       [-0.14719565, -0.07771419, -0.24006893, ...,  0.73498748,
        -0.66731899,  3.        ],
       [ 0.        ,  0.        ,  0.        , ..., -0.62216692,
        -0.68862028,  3.        ],
       [ 0.        ,  0.        ,  0.        , ..., -1.53288894,
        -1.50872026,  3.        ]]), 'labels': array([0., 0., 0., ..., 0., 0., 0.])},
      dtype=object)

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

X, y = load_data['122'].item()['features'], load_data['122'].item()['labels']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions on test data
y_pred_proba = model.predict_proba(X_test)[:, 1]

auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"AUC score: {auc_score:.4f}")

AUC score: 0.7495
